# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///InsertDatabaseName.db')
df  = pd.read_sql_table('df', engine)
df.iloc[10]

id                                                                       20
message                   There's nothing to eat and water, we starving ...
original                            Bon repo pa gen anyen menm grangou swaf
genre                                                                direct
related                                                                   1
request                                                                   1
offer                                                                     0
aid_related                                                               1
medical_help                                                              1
medical_products                                                          1
search_and_rescue                                                         0
security                                                                  0
military                                                                  0
child_alone 

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # Stem word tokens and remove stop words
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
target_names = list(y.columns.values)
target_names[3:]

['aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [8]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

In [9]:
y_train = np.array(y_train)
y_train[:,3]

array([0, 0, 0, ..., 1, 1, 0])

In [10]:
from sklearn.metrics import classification_report

In [11]:
#index [:,n] = : rows x n columns
for i in range(len(target_names)):
    print(classification_report(y_train[:,i],y_train_pred[:,i],target_names = target_names[i:]))

                        precision    recall  f1-score   support

               related       0.99      0.97      0.98      4569
               request       0.99      1.00      0.99     14951
                 offer       0.98      0.95      0.96       142

           avg / total       0.99      0.99      0.99     19662

                        precision    recall  f1-score   support

               request       0.99      1.00      0.99     16335
                 offer       1.00      0.93      0.96      3327

           avg / total       0.99      0.99      0.99     19662

                        precision    recall  f1-score   support

                 offer       1.00      1.00      1.00     19578
           aid_related       1.00      0.69      0.82        84

           avg / total       1.00      1.00      1.00     19662

                        precision    recall  f1-score   support

           aid_related       0.98      1.00      0.99     11519
          medical_help       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 35
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 34
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 33
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 32
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/met

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
%%time
# Create grid search object
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(performance_metric)

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 10)

# Find best parameters
tuned_model = cv.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.2345132743362832, total=  55.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.24397314617027768, total=  55.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.23680195300579798, total=  54.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.23619163869392737, total=  55.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 10.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.24992371071101618, total=  46.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.2445834604821483, total=  46.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.24107415318889228, total=  45.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.25556911809581934, total= 1.8min
[CV] clf__estimator__min_samples_split=2, cl

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 101.4min finished


CPU times: user 1h 41min 57s, sys: 53.7 s, total: 1h 42min 50s
Wall time: 1h 43min 17s


In [14]:
 # Get results of grid search
tuned_model.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([ 46.6807913 ,  38.05493776,  47.35383757,  37.63169018,
         94.37336755,  73.68182635,  96.60398428,  72.22613867,
         39.8387657 ,  35.01820978,  39.60995523,  34.2365884 ,
         77.67877801,  65.95480434,  77.31142735,  63.52029705,
         37.08175524,  33.65746156,  36.8060197 ,  32.62077888,
         71.09282613,  62.56681029,  70.03296932,  60.04147283]),
 'std_fit_time': array([ 0.41364341,  0.39637388,  0.39778841,  0.29248579,  0.36527074,
         0.85336863,  0.9378511 ,  0.63970601,  0.26526527,  0.2048204 ,
         0.42370445,  0.23771133,  0.45368231,  0.61123424,  0.36808448,
         0.45106361,  0.1633108 ,  0.31410345,  0.29708845,  0.10169513,
         0.27209947,  0.57780644,  0.47294195,  0.41915463]),
 'mean_score_time': array([  8.7205356 ,   8.2791911 ,   8.66091935,   8.25107718,
         11.11023474,  10.16365902,  11.03606558,  10.12564023,
          8.72014229,   8.3619016 ,   8.72749464,   8.34496037,
         11.1315

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
y_test = np.array(y_test)
tuned_pred_test = tuned_model.predict(X_test)
for i in range(len(target_names)):
    print(classification_report(y_test[:,i],tuned_pred_test[:,i],target_names = target_names[i:]))


                        precision    recall  f1-score   support

               related       0.66      0.43      0.52      1553
               request       0.84      0.93      0.88      4955
                 offer       0.35      0.33      0.34        46

           avg / total       0.79      0.81      0.79      6554

                        precision    recall  f1-score   support

               request       0.90      0.98      0.94      5407
                 offer       0.83      0.51      0.63      1147

           avg / total       0.89      0.90      0.89      6554

                        precision    recall  f1-score   support

                 offer       0.99      1.00      1.00      6520
           aid_related       0.00      0.00      0.00        34

           avg / total       0.99      0.99      0.99      6554

                        precision    recall  f1-score   support

           aid_related       0.80      0.81      0.81      3837
          medical_help       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 35
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 34
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 33
  .format(len(labels), len(target_na

In [21]:
dict1

' '

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [35]:
# Try using SVM instead of Random Forest Classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [1, 2, 3],
              'clf__estimator__C':[1, 10, 100]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, verbose = 10)

# Find best parameters
tuned_model2 = cv2.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


ValueError: The number of classes has to be greater than one; got 1

### 9. Export your model as a pickle file

In [33]:
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.